In [56]:
import itertools

import numpy as np
from pyomo.environ import *
from pyomo.environ import value as pyoval

## VERTEX ENUMERATION

In [57]:
# def vertex_enumerate(theta_nominal:list, theta_bounds:list) -> float:
#     vertex_list = list(itertools.product(['+','-'], repeat=len(theta_nominal)))
# 
#     if len(theta_nominal) != len(theta_bounds):
#         raise ValueError('Inconsistent input for nominal theta values and theta bounds')
#     else:
#         model_min = None
#         delta_min = None
#         active_set_list = None
#         vertex_min = None
# 
#         for vertex in vertex_list:
#             m = ConcreteModel()
#             m.i = Set(initialize=[0,1,2,3], ordered=True)
#             m.thetas = Var(m.i, within=NonNegativeReals)
#             m.delta = Var(within=NonNegativeReals)
#             m.z = Var(within=NonNegativeReals)
# 
#             m.f1 = Constraint(expr = -m.z/1.5 + m.thetas[1] -350 <= 0)
#             m.f2 = Constraint(expr = -0.5*m.z - 0.75*m.thetas[0] - m.thetas[1] - m.thetas[2] + 1388.5 <= 0)
#             m.f3 = Constraint(expr = m.z - 1.5*m.thetas[0] - 2*m.thetas[1] - m.thetas[2] - 2*m.thetas[3] + 2830 <= 0)
#             m.f4 = Constraint(expr = m.z - 1.5*m.thetas[0] - 2*m.thetas[1] - m.thetas[2] + 2044 <= 0)
#             m.f5 = Constraint(expr = -m.z + 1.5*m.thetas[0] + 2*m.thetas[1] + m.thetas[2] + 3*m.thetas[3] - 3153 <= 0)
# 
#             if vertex[0]=='+':
#                 m.theta0_UB = Constraint(expr = m.thetas[0] == theta_nominal[0] + theta_bounds[0][1]*m.delta)
#             elif vertex[0]=='-':
#                 m.theta0_LB = Constraint(expr = m.thetas[0] == theta_nominal[0] - theta_bounds[0][0]*m.delta)
# 
#             if vertex[1]=='+':
#                 m.theta1_UB = Constraint(expr = m.thetas[1] == theta_nominal[1] + theta_bounds[1][1]*m.delta)
#             elif vertex[1]=='-':
#                 m.theta1_LB = Constraint(expr = m.thetas[1] == theta_nominal[1] - theta_bounds[1][0]*m.delta)
# 
#             if vertex[2]=='+':
#                 m.theta2_UB = Constraint(expr = m.thetas[2] == theta_nominal[2] + theta_bounds[2][1]*m.delta)
#             elif vertex[2]=='-':
#                 m.theta2_LB = Constraint(expr = m.thetas[2] == theta_nominal[2] - theta_bounds[2][0]*m.delta)
# 
#             if vertex[3]=='+':
#                 m.theta3_UB = Constraint(expr = m.thetas[3] == theta_nominal[3] + theta_bounds[3][1]*m.delta)
#             elif vertex[3]=='-':
#                 m.theta3_LB = Constraint(expr = m.thetas[3] == theta_nominal[3] - theta_bounds[3][0]*m.delta)
# 
#             m.objective = Objective(expr=m.delta, sense=maximize)
#             solver = SolverFactory('gurobi')
#             solver.solve(m)
# 
#             if delta_min is None or pyoval(m.delta) < delta_min:
#                 delta_min = pyoval(m.delta)
#                 active_set_list = [c.name for c in [m.f1, m.f2, m.f3, m.f4, m.f5] if round(pyoval(c.body),5) == round(pyoval(c.upper),5)]
#                 model_min = m
#                 vertex_min = vertex
# 
#     return model_min, active_set_list, vertex_min, delta_min
# 
# theta_nominal = [620, 388, 583, 313]
# theta_bounds = [(10, 10), (10, 10), (10, 10), (10, 10)]
# 
# model_VE1, active_set_list1, vertex1, delta1 = vertex_enumerate(theta_nominal=theta_nominal, theta_bounds=theta_bounds)
# 
# print(f'Flexibility index calculated through vertex enumeration: {delta1}')
# print(f'Active set for vertex {vertex1}: {active_set_list1}')

In [58]:
def vertex_enumerate(A, b, theta_nominal:list, theta_bounds:list, num_design:int=0, num_control:int=0):
    vertex_list = list(itertools.product(['+','-'], repeat=len(theta_nominal)))

    if len(theta_nominal) != len(theta_bounds):
        raise ValueError('Inconsistent input for nominal theta values and theta bounds')
    else:
        model_min = None
        delta_min = None
        active_set_list = None
        vertex_min = None

        for vertex in vertex_list:
            m = ConcreteModel()
            
            m.Ai = RangeSet(A.shape[0])
            m.Aj = RangeSet(A.shape[1])
            m.i = RangeSet(num_design + num_control + len(theta_nominal))

            m.var = Var(m.i, within=NonNegativeReals) # structure (d,z,theta)
            m.delta = Var(within=NonNegativeReals)
            
            # system constraints: f(d,z,theta) <= 0
            def Ax_leq_b_rule(instance, i):
                return sum(A[i-1, j-1]*instance.var[j] for j in instance.Aj) <= b[i-1]
            m.Ax_leq_b = Constraint(m.Ai, rule=Ax_leq_b_rule)
            
            # theta constraints: theta = theta_nominal +- theta_bounds * delta
            m.constraints = ConstraintList()
            theta_index = num_design + num_control + 1
            for i in range(len(theta_nominal)):
                if vertex[i]=='+':
                    expr = m.var[theta_index] == theta_nominal[i] + theta_bounds[i][1]*m.delta
                else:
                    expr = m.var[theta_index] == theta_nominal[i] - theta_bounds[i][0]*m.delta
                m.constraints.add(expr)
                theta_index+=1
            
            # Objective
            m.objective = Objective(expr=m.delta, sense=maximize)
            solver = SolverFactory('gurobi')
            solver.solve(m)

            if delta_min is None or pyoval(m.delta) < delta_min:
                delta_min = pyoval(m.delta)
                active_set_list = [f'f{c}' for c in m.Ax_leq_b if round(pyoval(m.Ax_leq_b[c].body),5) == round(pyoval(m.Ax_leq_b[c].upper),5)]
                model_min = m
                vertex_min = vertex

    return model_min, active_set_list, vertex_min, delta_min

theta_nominal = [620, 388, 583, 313]
theta_bounds = [(10, 10), (10, 10), (10, 10), (10, 10)]

A = np.array([
    [-2/3, 0, 1, 0, 0],
    [-0.5, -0.75, -1, -1, 0],
    [1, -1.5, -2, -1, -2],
    [1, -1.5, -2, -1, 0],
    [-1, 1.5, 2, 1, 3]
])

b = np.array([350, -1388.5, -2830, -2044, 3153])

model_VE2, active_set_list2, vertex2, delta2 = vertex_enumerate(A=A, b=b, num_control=1, theta_nominal=theta_nominal, theta_bounds=theta_bounds)

print(f'Flexibility index calculated through vertex enumeration: {delta2}')
print(f'Active set for vertex {vertex2}: {active_set_list2}')

Flexibility index calculated through vertex enumeration: 0.5600000000000023
Active set for vertex ('-', '-', '-', '-'): ['f1', 'f3']


## ACTIVE SET STRATEGY

In [ ]:
def active_set_method(A, b, theta_nominal:list, theta_bounds:list, num_design:int=0, num_control:int=0):
    if len(theta_nominal) != len(theta_bounds):
        raise ValueError('Inconsistent input for nominal theta values and theta bounds')
    else:
        m = ConcreteModel()
        
        m.Ai = RangeSet(A.shape[0])
        m.Aj = RangeSet(A.shape[1])
        m.i = RangeSet(num_design + num_control + len(theta_nominal))

        m.var = Var(m.i, within=NonNegativeReals) # structure (d,z,theta, s, y, dual)
        m.delta = Var(within=NonNegativeReals)

In [59]:
# def active_set(theta_nominal:list, theta_bounds:list, bigM:float = 1e3):
#     m = ConcreteModel()
# 
#     m.j = Set(initialize=[1,2,3,4,5], ordered=True)
#     m.t = Set(initialize=[0,1,2,3], ordered=True)
# 
#     m.z = Var(within=NonNegativeReals)
#     m.T = Var(m.t, within=NonNegativeReals)
#     m.slack = Var(m.j, within=NonNegativeReals)
#     m.dual = Var(m.j, within=NonNegativeReals)
#     m.y = Var(m.j, within=Binary)
#     m.delta = Var(within=NonNegativeReals)
# 
#     m.f1 = Constraint(expr= -m.z/1.5 + m.T[1] -350 + m.slack[1] == 0)
#     m.f2 = Constraint(expr = -0.5*m.z - 0.75*m.T[0] - m.T[1] - m.T[2] + 1388.5 + m.slack[2] == 0)
#     m.f3 = Constraint(expr = m.z - 1.5*m.T[0] - 2*m.T[1] - m.T[2] - 2*m.T[3] + 2830 + m.slack[3] == 0)
#     m.f4 = Constraint(expr = m.z - 1.5*m.T[0] - 2*m.T[1] - m.T[2] + 2044 + m.slack[4] == 0)
#     m.f5 = Constraint(expr = -m.z + 1.5*m.T[0] + 2*m.T[1] + m.T[2] + 3*m.T[3] - 3153 + m.slack[5] == 0)
# 
#     m.dual_sum = Constraint(expr = sum(m.dual[i] for i in m.j) == 1)
#     m.df_dz = Constraint(expr = -m.dual[1]/1.5 + 0.5*m.dual[2] + m.dual[3] + m.dual[4] - m.dual[5] == 0)
# 
#     def complementary_rule(instance, j):
#         return instance.slack[j] - bigM*(1-instance.y[j]) <= 0
#     m.complementary = Constraint(m.j, rule=complementary_rule)
# 
#     def dual_UB_rule(instance, j):
#         return instance.dual[j] - instance.y[j] <= 0
#     m.dual_UB = Constraint(m.j, rule=dual_UB_rule)
# 
#     m.max_active_set = Constraint(expr = sum(m.y[j] for j in m.j) == len(list(m.z.keys())) + 1)
# 
#     def T_LB_rule(instance, t):
#         return theta_nominal[t] - m.delta * theta_bounds[t][0] - instance.T[t] <= 0
#     m.T_LB = Constraint(m.t, rule=T_LB_rule)
# 
#     def T_UB_rule(instance, t):
#         return instance.T[t] - theta_nominal[t] + m.delta * theta_bounds[t][1] <= 0
#     m.T_UB = Constraint(m.t, rule=T_UB_rule)
# 
#     m.objective = Objective(expr=m.delta, sense=minimize)
# 
#     solver = SolverFactory('gurobi')
#     solver.solve(m)
# 
#     active_set_list = [getattr(m, f'f{j}').name for j in m.j if pyoval(m.dual[j])]
# 
#     return m, active_set_list, pyoval(m.delta)
# 
# theta_nominal = [620, 388, 583, 313]
# theta_bounds = [(10, 10), (10, 10), (10, 10), (10, 10)]
# 
# model_AS, active_set_list, flexibility_index = active_set(theta_nominal=theta_nominal, theta_bounds=theta_bounds, bigM=700)
# 
# print(f'Flexibility index calculated through active set strategy: {flexibility_index}')
# print(f'Active set: {active_set_list}')